In [ ]:
import cdsapi
import healpy as hp
from datetime import datetime, timedelta
import xarray as xr
import numpy as np
from pathlib import Path
import zarr

#additional packages that are required to run this project are h5netcdf, netcdf4, scipy, zarr

In [ ]:
# CONFIG

#====== download config ======#
collection='reanalysis-era5-pressure-levels' #this notebook was only tested for the specified collection

variables=['specific_humidity'] # other variables can be found at https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation#ERA5:datadocumentation-Table9. This notebook was only developed for continues pressure level variables

pressure_levels = ['300', '500', '800', '900', '975']

temporal_extend = {
    "start": '2024-12-01',
    "end": '2024-12-05',
    "times": ['00:00:00', '06:00:00', '12:00:00', '18:00:00'],
}

spacial_extend = 'global' #for a limited area provide an array of shape [north, west, south, east]. north, south of range [-90, 90] west, east [-180, 180]

data_format = 'netcdf' # Downstream Processing requires netcdf so only change this variable if you want to change the processing pipeline

grid = ['0.25', '0.25'] # [res_long, res_lat] resolution of the grid for the download. The specified 0.25° are the default resolution for atmospheric data in the ERA5 Dataset

#====== storage config =======#
collection_short_name='era5' #short name used for data_storage
storage_path='../data/'
unprocessed_path=f'{collection_short_name}_regular_grid/'
zarr_file=f'{collection_short_name}_healpix.zarr'

#===== regridding config =====#

nsides = [8, 16]
interpolation_method = 'linear' # methode for the interpolation to healpix (one of "linear", "nearest", "zero", "slinear", "quadratic", "cubic", "quintic", "polynomial")

In [ ]:
if not Path(storage_path).exists():
    Path(storage_path).mkdir()

if not Path(storage_path + unprocessed_path).exists():
    Path(storage_path + unprocessed_path).mkdir()

In [ ]:
# This cell iterates over the time period defined in temporal_extend and creates a dict containing day month and year for each day in the interval. This will later be used for the main downloading loop
start_date = datetime.strptime(temporal_extend["start"], '%Y-%m-%d')
end_date = datetime.strptime(temporal_extend["end"], '%Y-%m-%d')

time_chunks = []
for delta in range((end_date - start_date).days + 1): # iterate over the date range to create a dict for each day in our time_chunks list. Later we can loop over those list entries to loop over the days
    i_date = start_date + timedelta(days=delta)
    time_chunks.append({
        "day": i_date.day,
        "month": i_date.month,
        "year": i_date.year,
        "string": i_date.strftime("%Y-%m-%d")
    })

time_chunks[:3] # :3 needed to limit console size for bigger temporal extends

In [ ]:
# extract the calculations for the healpix grids out of the main routine, so we don't do them multiple times (potentially for a lot of days / nside resolutions, amounting to a lot of calculations)
hp_grids = []

for nside in nsides: # iterate over nside resolutions and create the grid so we don't have to do it multiple tim
    n_pix = hp.nside2npix(nside) # calculate number of pixels
    theta, phi = hp.pix2ang(nside, np.arange(n_pix)) # calculate the pixel centers for each pixel in rad

    hp_latitude = 90 - np.degrees(theta) # convert to degrees (North Pole = 0, South Pole = 180) and subtract from 90 to get to (North Pole = 90, South Pole = -90)
    hp_longitude = np.degrees(phi) # convert to degrees (Greenwich = 0, increasing eastwards)

    hp_grids.append({"latitude": hp_latitude, "longitude": hp_longitude, "name": f'nside-{nside}'})

In [ ]:
c = cdsapi.Client()

zarr_path = storage_path + zarr_file

for time_chunk in time_chunks:
    file_name = f'{collection_short_name}_{time_chunk["string"]}'
    complete_path = storage_path + unprocessed_path + file_name

    c.retrieve(collection, {
        "product_type": ['reanalysis'],
        "variable": variables,
        "year": time_chunk["year"],
        "month": time_chunk["month"],
        "day": time_chunk["day"],
        "time": temporal_extend["times"],
        "pressure_level": pressure_levels,
        "grid": grid,
        "area": spacial_extend,
       "data_format": data_format,
    }, complete_path)

    ds = xr.open_dataset(complete_path)
    ds = ds.sortby('latitude')

    for hp_grid in hp_grids:
        ds_hp = ds.interp(
            latitude=xr.DataArray(hp_grid["latitude"]),
            longitude=xr.DataArray(hp_grid["longitude"]),
            method=interpolation_method
        )

        group_path = hp_grid["name"]

        if not Path(zarr_path).exists() or group_path not in zarr.open_group(zarr_path, mode="a").group_keys() or "time" not in xr.open_zarr(zarr_path, group=group_path).dims:
            ds.to_zarr(zarr_path, group=group_path, mode="w")
        else:
            ds.to_zarr(zarr_path, group=group_path, mode="a", append_dim="time")